In [2]:
# basics
import utils
import numpy as np
from tqdm.notebook import tqdm 


# torch
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter



# custom modules
from data_acquisition import DataHandler
from data_preparation import apply_preprocessing_pipeline


# Configure logging for the pipeline
logger = utils.setup_logger(level='ERROR')

In [3]:
cities = ['London']#, 'CapeTown', 'Hamburg', 'Johannesburg', 'London', 'Montreal', 'Paris', 'Seoul', 'Singapore', 'Sydney']

datahandler = DataHandler(logger)


In [4]:
# load images and mask for all specified cites

images = []
masks=[]
for city in tqdm(cities):
    images.append(datahandler.get_satellite_image(city))
    masks.append(datahandler.get_building_mask(city, all_touched=False))

  0%|          | 0/1 [00:00<?, ?it/s]

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as 

In [5]:
# apply training pipeline
# TODO make train test split consistent so we can train with multiple sizes, dont know if there is an advantage though
train_loader, test_loader = apply_preprocessing_pipeline(images, masks, patch_size = 128, train_ratio = 0.8, batch_size = 64)

In [6]:
# initialize model, taken from exercise pdf
model = nn.Sequential(
    nn.Conv2d(6, 32, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(128, 1, kernel_size=1, padding=0),
    nn.Sigmoid())

# initialize tensorboard writer
writer = SummaryWriter()

In [8]:
import os
import torch
from torch import nn
import lightning as L
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [10]:
sample = next(iter(train_loader))   
sample.shape

torch.Size([64, 7, 128, 128])

In [16]:
type(sample), sample.dtype

(torch.Tensor, torch.uint16)

In [26]:
class convNetSimple(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
                nn.Conv2d(6, 32, kernel_size=3, padding=1), nn.ReLU(),
                nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
                nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
                nn.Conv2d(128, 1, kernel_size=1, padding=0),
                nn.Sigmoid())
    
    def forward(self, x):
        return self.model(x)

In [27]:
class LitNet(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = nn.BCELoss()

    def training_step(self, batch, batch_idx):
        x, y = batch[:,:-1], batch[:,-1]
        outs = self.model(x.float())
        loss = self.loss(outs, y.unsqueeze(1).float())
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch[:,:-1], batch[:,-1]
        outs = self.model(x.float())
        loss = self.loss(outs, y.unsqueeze(1).float())
        
        values = {
            "test_loss": loss,
        
        }
        self.log_dict(values, on_epoch=True, on_step=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [28]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint


L.seed_everything(42)
convmodel = LitNet(convNetSimple())
trainer = L.Trainer(
    default_root_dir="models",
    # callbacks=[
    #     EarlyStopping(
    #         monitor="val_loss",
    #         mode="min",
    #         patience=10,
    #     )
    #     ModelCheckpoint(
    #         monitor="val_loss",
    #         mode="min",
    #         save_top_k=2,
    #         dirpath="models",
    #         filename="best_model"
    #     )
    # ]
    # val_check_interval=1,
    fast_dev_run=False,
    # num_sanity_val_steps=2,
    max_epochs=40,
    log_every_n_steps=20,
)

# training
trainer.fit(convmodel, 
    train_dataloaders=train_loader,
    # val_dataloaders=val_loader   #### HIER brauchen wir noch einen validation loader
)

# testing
trainer.test(
    convmodel,
    dataloaders=test_loader
)


INFO: Seed set to 42
2024-06-28 22:59:42,313 - lightning.fabric.utilities.seed - INFO - seed_everything - Seed set to 42
INFO: GPU available: True (cuda), used: True
2024-06-28 22:59:42,339 - lightning.pytorch.utilities.rank_zero - INFO - _info - GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
2024-06-28 22:59:42,341 - lightning.pytorch.utilities.rank_zero - INFO - _info - TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
2024-06-28 22:59:42,342 - lightning.pytorch.utilities.rank_zero - INFO - _info - HPU available: False, using: 0 HPUs
2024-06-28 22:59:42,344 - lightning.pytorch.loggers.tensorboard - WARNING - _get_next_version - Missing logger folder: models/lightning_logs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2024-06-28 22:59:42,347 - lightning.pytorch.accelerators.cuda - INFO - set_nvidia_flags - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type          | Params | Mode 
---------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=40` reached.
2024-06-28 23:02:21,949 - lightning.pytorch.utilities.rank_zero - INFO - _info - `Trainer.fit` stopped: `max_epochs=40` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2024-06-28 23:02:21,978 - lightning.pytorch.accelerators.cuda - INFO - set_nvidia_flags - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        5.8461079597473145
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 5.8461079597473145}]

In [29]:
# # Instantiate the model, loss function, and optimizer
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# # Training loop
# num_epochs = 50

# model.train()
# for epoch in tqdm(range(num_epochs)):
#     for batch in train_loader:
#         # splid in inputs and labels
#         inputs = batch[:,:-1].to(torch.float32)
#         labels = batch[:,-1, np.newaxis].to(torch.float32)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward pass
#         outputs = model(inputs)

#         # calculate loss
#         loss = criterion(outputs, labels)

#         # write to tensorboard
#         writer.add_scalar("Loss/train", loss, epoch)

#         # backward pass
#         loss.backward()

#         # optimizer step
#         optimizer.step()
    


## Save Model

In [30]:
# import os

# os.makedirs("saved_models", exist_ok=True)
# torch.save(model.state_dict(), "saved_models/model1")

# Evaluation

In [31]:
# t  = torch.Tensor(test_loader.dataset)

# # splid in inputs and labels
# test_inputs = t[:,:-1]#.to(torch.float32)
# test_labels = t[:,-1, np.newaxis]#.to(torch.float32)

# test_results = model(test_inputs).detach()

# # see how many percnet where predicted right
# threshold = 0.5
# ((test_results>threshold)==test_labels).sum()/np.prod(test_labels.shape)



In [32]:
# from sklearn.metrics import RocCurveDisplay

# RocCurveDisplay.from_predictions(
#    test_labels.flatten(), test_results.flatten())

In [ ]:
writer.flush()

# Download

In [ ]:


buildings = []
sat_images = []
building_masks = []

for city in cities: 
    buildings.append(datahandler.get_buildings(city))
    sat_images.append(datahandler.get_satellite_image(city))
    building_masks.append(datahandler.get_building_mask(city))

# Plot the expected results for the first city 
datahandler.plot(city[0])

In [ ]:
import data_preparation

for city in cities:
    data_preparation.create_tensor(city)

# Download

In [ ]:
# Download 

for city in cities: 
    sat_image = datahandler.get_satellite_image(city)
    mask = datahandler.get_building_mask(city)

# Plot the expected results for the first city 
datahandler.plot(city[0])